In [42]:
import pandas as pd
import time
from tqdm import tqdm
import gc
import warnings 
warnings.filterwarnings('ignore')

# 讀取資料

In [27]:
s = time.time()
# 訓練集
train_data = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
# 測試集
test_data = pd.read_feather('../data/需預測的顧客名單及提交檔案範例.feather')
print('cost time:',time.time()-s)
# 展示
print(train_data.shape,test_data.shape)
display(train_data.head())
display(test_data.head())

cost time: 23.545891761779785
(32975654, 53) (500000, 4)


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


# 定義欄位

In [28]:
print(gc.collect())
feature_columns = list(set(train_data.columns.tolist()) - set(['chid','shop_tag']))
categorical_features = ['masts','educd','trdtp','naty','poscd','cuorg','gender_code','age','primary_card']
numerical_features = list(set(feature_columns)-set(categorical_features))
train_df = train_data
sample_submit = test_data
len(feature_columns),len(categorical_features),len(numerical_features)

706


(51, 9, 42)

# 特徵工程

In [43]:
# 從source_df抓對應的chid算一些統計資訊當作特徵併入target_df裡面
def feature_engineer(target_df,source_df):
    for idx in tqdm(target_df.index):
        # 取出這一條row的chid
        chid = target_df.loc[idx,'chid']
        # 從source_df取出所有屬於這個chid的row
        group_by_chid = source_df.loc[source_df.chid==chid]
        # 數值特徵(numerical_features)統計資訊
        for col in numerical_features: 
            target_df.loc[idx,f"mean_of_{col}"] = group_by_chid.loc[:,col].mean()
            target_df.loc[idx,f"max_of_{col}"] = group_by_chid.loc[:,col].max()
            target_df.loc[idx,f"min_of_{col}"] = group_by_chid.loc[:,col].min()
        # 類別特徵(categorical_features)統計資訊
        for col in categorical_features:
            # 1.nunique (這個欄位有多少種不一樣的數值)
            target_df.loc[idx,f"nunique_of_{col}"] = group_by_chid.loc[:,col].nunique()
            # 2.norm_count (這個欄位在每一種數值上出現的次數(count)除以總數(normalize)等於取平均(mean))
            cat_feat = pd.get_dummies(group_by_chid.loc[:,col],columns=col) # one_hot 的 np.array
            cat_feat = pd.DataFrame(cat_feat.mean(axis=0)).T # 轉成DataFrame取平均
            cat_feat.index = [idx] # 處理DataFrame格式
            cat_feat.columns = [ "norm_count_" + col + "_" + str(i) for i in cat_feat.columns] # 處理DataFrame格式
            try:
                target_df = target_df.join(cat_feat) # 合併DataFrame
            except:
                target_df.loc[idx,cat_feat.columns] = cat_feat.values.reshape(-1)
    return target_df

In [46]:
sample_submit_feature = feature_engineer(sample_submit.sample(1000),train_data)
train_data_feature = feature_engineer(train_data.sample(1000),train_data)

100%|██████████| 1000/1000 [01:25<00:00, 11.70it/s]


In [62]:
sample_submit_new_features = sample_submit_feature.columns.tolist()[4:]

In [64]:
common_features = set(sample_submit_new_features)&set(train_data_feature.columns)

In [68]:
print(sample_submit_feature[common_features].shape)
print(train_data_feature[common_features].shape) 
display(sample_submit_feature[common_features])
display(train_data_feature[common_features])


(1000, 190)
(1000, 190)


,norm_count_trdtp_12.0,max_of_card_9_txn_cnt,norm_count_trdtp_8.0,nunique_of_educd,norm_count_age_7.0,min_of_card_6_txn_cnt,mean_of_card_8_txn_amt_pct,mean_of_card_4_txn_cnt,norm_count_gender_code_0.0,norm_count_poscd_4.0,...,norm_count_cuorg_18.0,norm_count_poscd_5.0,norm_count_educd_2.0,mean_of_card_14_txn_amt_pct,max_of_txn_cnt,min_of_card_3_txn_cnt,norm_count_trdtp_5.0,mean_of_card_9_txn_amt_pct,norm_count_cuorg_6.0,nunique_of_age
403024,NaN,0.0,NaN,1.0,NaN,1.0,0.000000,0.000000,1.0,NaN,...,NaN,NaN,NaN,0.0,2.0,0.0,1.0,0.0,NaN,1.0
291704,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,1.653846,1.0,NaN,...,NaN,NaN,NaN,0.0,5.0,0.0,NaN,0.0,NaN,1.0
61007,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.0,7.0,0.0,NaN,0.0,NaN,1.0
391366,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.0,3.0,0.0,NaN,0.0,NaN,1.0
194841,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,1.733333,1.0,NaN,...,NaN,NaN,NaN,0.0,3.0,0.0,NaN,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357637,NaN,0.0,NaN,1.0,NaN,0.0,0.859485,0.000000,1.0,NaN,...,NaN,NaN,NaN,0.0,9.0,0.0,NaN,0.0,NaN,1.0
162916,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,0.000000,1.0,NaN,...,NaN,NaN,1.0,0.0,3.0,0.0,NaN,0.0,NaN,1.0
196567,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,1.321429,1.0,NaN,...,NaN,NaN,NaN,0.0,3.0,0.0,NaN,0.0,NaN,1.0
59267,NaN,0.0,NaN,1.0,1.0,0.0,0.000000,1.625000,NaN,NaN,...,NaN,NaN,NaN,0.0,3.0,0.0,NaN,0.0,NaN,1.0


,norm_count_trdtp_12.0,max_of_card_9_txn_cnt,norm_count_trdtp_8.0,nunique_of_educd,norm_count_age_7.0,min_of_card_6_txn_cnt,mean_of_card_8_txn_amt_pct,mean_of_card_4_txn_cnt,norm_count_gender_code_0.0,norm_count_poscd_4.0,...,norm_count_cuorg_18.0,norm_count_poscd_5.0,norm_count_educd_2.0,mean_of_card_14_txn_amt_pct,max_of_txn_cnt,min_of_card_3_txn_cnt,norm_count_trdtp_5.0,mean_of_card_9_txn_amt_pct,norm_count_cuorg_6.0,nunique_of_age
22459315,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,0.000000,1.0,NaN,...,NaN,NaN,NaN,0.0,6.0,0.0,1.0,0.0,NaN,1.0
31466410,NaN,0.0,NaN,1.0,NaN,0.0,0.013793,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.0,11.0,0.0,1.0,0.0,NaN,1.0
13342988,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,0.000000,1.0,NaN,...,NaN,NaN,NaN,0.0,10.0,0.0,NaN,0.0,NaN,1.0
9606297,0.291667,0.0,NaN,1.0,NaN,0.0,0.000000,2.773810,NaN,NaN,...,NaN,NaN,NaN,0.0,15.0,0.0,NaN,0.0,NaN,1.0
16495815,NaN,0.0,NaN,1.0,NaN,0.0,0.088259,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.0,18.0,0.0,NaN,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6134645,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.0,7.0,0.0,NaN,0.0,NaN,1.0
3343544,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,0.000000,1.0,NaN,...,NaN,NaN,NaN,0.0,18.0,0.0,NaN,0.0,NaN,1.0
2612040,NaN,0.0,NaN,1.0,NaN,0.0,0.000000,3.758621,1.0,NaN,...,NaN,NaN,NaN,0.0,15.0,0.0,NaN,0.0,NaN,1.0
17331199,1.000000,0.0,NaN,1.0,NaN,0.0,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,0.0,12.0,0.0,NaN,0.0,NaN,1.0


In [69]:
train_data_feature.isnull().sum().sum()

54966

In [70]:
sample_submit_feature.isnull().sum().sum()

54947

# 保存

In [71]:
train_data_feature.reset_index().to_feather('../data/train_data_feature.feather')
sample_submit_feature.reset_index().to_feather('../data/sample_submit_feature.feather')